In [48]:
#import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from fancyimpute import KNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [ ]:
absenteeism = pd.read_csv("Absenteeism_at_work_train.csv")

print(absenteeism.head())


In [ ]:
absenteeism.describe()

In [ ]:

absenteeism.info()
# Age and Work load Average/day need to be converted from object to Integer

In [ ]:
absenteeism.dtypes

In [ ]:
# Change to categorical

absenteeism['ID'] = absenteeism['ID'].astype('category')
absenteeism['Reason for absence'] = absenteeism['Reason for absence'].astype('category')
absenteeism['Month of absence'] = absenteeism['Month of absence'].astype('category')
absenteeism['Day of the week'] = absenteeism['Day of the week'].astype('category')
absenteeism['Seasons'] = absenteeism['Seasons'].astype('category')
absenteeism['Disciplinary failure'] = absenteeism['Disciplinary failure'].astype('category')
absenteeism['Education'] = absenteeism['Education'].astype('category')
absenteeism['Son'] = absenteeism['Son'].astype('category')
absenteeism['Social drinker'] = absenteeism['Social drinker'].astype('category')
absenteeism['Social smoker'] = absenteeism['Social smoker'].astype('category')
absenteeism['Pet'] = absenteeism['Pet'].astype('category')

In [ ]:
# Check missing values

absenteeism.isnull().sum()

# Weight and Hit target has missing values

In [ ]:
# Drop NAs

absenteeism = absenteeism.dropna(axis = 0)

absenteeism.isnull().sum()

In [ ]:
# Check Age column values

# We see 'R' as a value for Age in a row. Update it to 0
absenteeism.loc[absenteeism.Age == 'R', "Age"] = 0

# Convert the column to numeric
absenteeism[['Age']] = absenteeism[['Age']].apply(pd.to_numeric)



In [ ]:
# To convert 'Work load Average/day' column from object to numeric remove the commas in the values

absenteeism.replace(',','', regex=True, inplace=True)

# Convert the column to numeric
absenteeism["Work load Average/day "] = pd.to_numeric(absenteeism["Work load Average/day "], errors='ignore')

# Renaming the column name to remove the blank space 
absenteeism.rename(columns = {'Work load Average/day ':'Work_load_Average'}, inplace = True)



In [ ]:
# categorising the independdent variables in to category " Continuos" and "Categorical"
continuous_vars = ['Distance from Residence to Work', 'Service time', 'Age', 'Work_load_Average', 'Transportation expense',
       'Hit target', 'Weight', 'Height', 'Body mass index', 'Absenteeism time in hours']

categorical_vars = ['ID','Reason for absence','Month of absence','Day of the week',
                     'Seasons','Disciplinary failure', 'Education', 'Social drinker',
                     'Social smoker', 'Pet', 'Son']

In [ ]:
# Outlier check using boxplot

for i in continuous_vars:
    if i == 'Absenteeism time in hours':
        continue
    sns.boxplot(data = absenteeism[i], orient="h", palette="Set2")
    plt.title("Checking Outlier for Variable "+str(i))
    plt.show()
 
# We can see outliers in Service time,Age, wor_load_Average per day, Transportation expense, Hit target, Weight, Height, Body Mass index,


In [ ]:
# Outlier treatement

# Identify the continuous independent variable with outlier and replace the outliers with the lower bound and upper bound values

# We are treating outliers in all the continous variables except the dependent variable
# The outlier in the dependent variable could be inflencial

for col in continuous_vars : 
    if col == 'Absenteeism time in hours':
        continue
    q1 = absenteeism[col].quantile(.25)
    q3 = absenteeism[col].quantile(.75)
    OQR = q3 - q1
    lb = q1-(q3-q1)*1.5
    ub = q3+(q3-q1)*1.5
    absenteeism[col] = np.where(absenteeism[col] < lb, lb,absenteeism[col])
    absenteeism[col] = np.where(absenteeism[col] > ub, ub,absenteeism[col])



In [ ]:
# Check outliers post outlier treatment

for i in continuous_vars:
    if i == 'Absenteeism time in hours':
        continue
    sns.boxplot(data = absenteeism[i], orient="h", palette="Set2")
    plt.title("Checking Outlier for Variable "+str(i))
    plt.show()
 
# plot shows that the outliers are updated

In [ ]:
# Check the correlation between the variables using heatmap

plt.figure(figsize = (20,6))
sns.heatmap(absenteeism.corr(), annot = True)
plt.show()

# With the help of correlation plot we can see that Body Mass Index and Weight has high correlation near to 1. 
# Which determines that we can drop one of the variable from the dataset. 
# Similarly Service time and Age has a significant correlation too

In [ ]:
# Check the bar graph of categorical Data using factorplot
sns.set_style("whitegrid")
sns.factorplot(data=absenteeism, x='Reason for absence', kind= 'count',size=4,aspect=2)
sns.factorplot(data=absenteeism, x='Seasons', kind= 'count',size=4,aspect=2)
sns.factorplot(data=absenteeism, x='Education', kind= 'count',size=4,aspect=2)
sns.factorplot(data=absenteeism, x='Disciplinary failure', kind= 'count',size=4,aspect=2)

In [ ]:
plt.hist(data=absenteeism, x='Weight', bins='auto', label='Weight')
plt.xlabel('Weight')
plt.title("Weight Distribution")

In [ ]:
# Check the distribution of numerical data using histogram
plt.hist(data=absenteeism, x='Age', bins='auto', label='Age')
plt.xlabel('Age')
plt.title("Age Distribution")

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)
N = 663
x = absenteeism['Age']
y = absenteeism['Absenteeism time in hours']
colors = np.random.rand(N)
plt.scatter(x, y, c=colors, alpha=0.2)
plt.xlabel("Age")
plt.ylabel("Absenteeism time in hours")
plt.title("Distribution of Absenteeism time by Age")
plt.show()

In [ ]:
x = absenteeism['Reason for absence']
y = absenteeism['Absenteeism time in hours']
plt.scatter(x, y, alpha=0.5)
plt.xlabel("Reason for Absence")
plt.ylabel("Absenteeism time in hours")
plt.title("Distribution of Absenteeism time by Reason of Absence")
plt.show()

In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)
N = 663
x = absenteeism['Day of the week']
y = absenteeism['Absenteeism time in hours']
colors = np.random.rand(N)
plt.scatter(x, y, c=colors, alpha=0.2)
plt.xlabel("Day of the week")
plt.ylabel("Absenteeism time in hours")
plt.title("Distribution of Absenteeism time by Day of the week")
plt.show()

## Feature Selection

In [ ]:
# Get dataframe with all continuous variables
absenteeism_corr = absenteeism.loc[:,continuous_vars]

In [ ]:
# Check for multicollinearity using corelation graph
# Set the width and hieght of the plot
f, ax = plt.subplots(figsize=(10, 10))

# Generate correlation matrix
corr = absenteeism_corr.corr()

# Plot using seaborn library
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), 
            cmap=sns.diverging_palette(220, 50, as_cmap=True),
            square=True, ax=ax, annot = True)
plt.plot()

In [ ]:
# Variable Reduction
to_drop = ['Weight']
absenteeism = absenteeism.drop(to_drop, axis = 1)

In [ ]:
# Updating the Continuous and Categorical Variables
continuous_vars.remove('Weight')

In [ ]:
# Make a copy of clean data and export it as excel file
clean_data = absenteeism.copy()

In [ ]:
continuous_vars

## Feature Scaling

In [ ]:
# Normality check
for i in continuous_vars:
    if i == 'Absenteeism time in hours':
        continue
    sns.distplot(absenteeism[i],bins = 'auto')
    plt.title("Checking Distribution for Variable "+str(i))
    plt.ylabel("Density")
    plt.show()

In [ ]:
# Normalization of continuous variables

for i in continuous_vars:
    if i == 'Absenteeism time in hours':
        continue
    absenteeism[i] = (absenteeism[i] - absenteeism[i].min())/(absenteeism[i].max()-absenteeism[i].min())
    
    
print(absenteeism.head())

## Machine Learning Models

In [32]:
# Create dummy variables of factor variables
absenteeism = pd.get_dummies(data = absenteeism, columns = categorical_vars)

# Copying dataframe
absenteeism_copy = absenteeism.copy()

In [33]:
# Get number of rows and columns
absenteeism.shape

(663, 113)

In [34]:
# Observe the first row
absenteeism.head(1)

,Transportation expense,Distance from Residence to Work,Service time,Age,Work_load_Average,Hit target,Height,Body mass index,Absenteeism time in hours,ID_1,...,Pet_1,Pet_2,Pet_4,Pet_5,Pet_8,Son_0,Son_1,Son_2,Son_3,Son_4
0,0.714506,0.659574,0.470588,0.430556,0.350753,0.806452,0.625,0.578947,4,0,...,1,0,0,0,0,0,0,1,0,0


In [39]:
# Splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(
    absenteeism.iloc[:, absenteeism.columns != 'Absenteeism time in hours'], 
    absenteeism.iloc[:, 8], test_size = 0.20, random_state = 1
)

In [42]:
# Utility

def calculate_root_mean_square_error(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

## Decision Tree with regression

In [56]:
# Build decsion tree using DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor(random_state = 1).fit(X_train, y_train)

# Perdict for test cases
decision_tree_predictions = decision_tree_model.predict(X_test)

# Create data frame for actual and predicted values
absenteeism_decision_tree = pd.DataFrame({'actual': y_test, 'pred': decision_tree_predictions})
print(absenteeism_decision_tree.head())

# Calculate RMSE and R-squared value
print("Root Mean Squared Error: {}".format(str(calculate_root_mean_square_error(y_test, decision_tree_predictions))))
print("R#^2 Score(coefficient of determination): {}".format(str(r2_score(y_test, decision_tree_predictions))))

# print("Confusion matrix: {}".format(confusion_matrix(y_test, decision_tree_predictions)))
# print("Accuracy score is: {}".format(accuracy_score(y_test, decision_tree_predictions) * 100))

     actual  pred
328       2   3.0
148       1   2.0
47        4   4.0
301      16   8.0
507       1   3.0
Root Mean Squared Error: 11.340674682559195
R#^2 Score(coefficient of determination): 0.16924356555823605


## Random Forest with regression

In [59]:
# Build random forest using RandomForestRegressor
random_forest_model = RandomForestRegressor(n_estimators = 500, random_state = 1).fit(X_train,y_train)

# Perdict for test cases
random_forest_model_predictions = random_forest_model.predict(X_test)

# Create data frame for actual and predicted values
absenteeism_random_forest = pd.DataFrame({'actual': y_test, 'pred': random_forest_model_predictions})
print(absenteeism_random_forest.head())

# Calculate RMSE and R-squared value
print("Root Mean Squared Error: {}".format(
    str(calculate_root_mean_square_error(y_test, random_forest_model_predictions)))
)
print("R#^2 Score(coefficient of determination): {}".format(str(r2_score(y_test, random_forest_model_predictions))))

# print("Confusion matrix: {}".format(confusion_matrix(y_test, random_forest_model_predictions)))
# print("Accuracy score is: {}".format(accuracy_score(y_test, random_forest_model_predictions) * 100))

     actual       pred
328       2   3.125000
148       1   3.430967
47        4   2.846533
301      16  21.980000
507       1   5.848000
Root Mean Squared Error: 11.028831792183091
R#^2 Score(coefficient of determination): 0.2143032533699346


## Dimension Reduction using PCA

In [ ]:
# Get the target variable
target = absenteeism['Absenteeism time in hours']

In [ ]:
# Get the number of rows and columns of data
absenteeism.shape

In [ ]:
# Converting data to numpy array
X = absenteeism.values

# Data has 116 variables so no of components of PCA = 115
pca = PCA(n_components=115)
pca.fit(X)

# Proportion of variance explained
e xplained_variance_ratio = pca.explained_variance_ratio_

#Cumulative scree plot
cum_sum = np.cumsum(np.round(explained_variance_ratio, decimals=4)*100)

# Draw the plot
plt.plot(cum_sum)
plt.show()

In [ ]:
# Selecting 45 components since it explains almost 95+ % data variance
pca = PCA(n_components=45)

# Fitting the selected components to the data
pca.fit(X)

#S plitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state = 1)

## Decision Tree with regression using PCA

In [60]:
# Build decsion tree using DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor(random_state=1).fit(X_train,y_train)

# Perdict for test cases
decision_tree_predictions = decision_tree_model.predict(X_test)

# Create data frame for actual and predicted values
absenteeism_decision_tree = pd.DataFrame({'actual': y_test, 'pred': decision_tree_predictions})
print(absenteeism_decision_tree.head())

# Calculate RMSE and R-squared value
print("Root Mean Squared Error: {}".format(str(calculate_root_mean_square_error(y_test, decision_tree_predictions))))
print("R^2 Score(coefficient of determination): {}".format(str(r2_score(y_test, decision_tree_predictions))))

# print("Confusion matrix: {}".format(confusion_matrix(y_test, decision_tree_predictions)))
# print("Accuracy score is: {}".format(accuracy_score(y_test, decision_tree_predictions) * 100))

     actual  pred
328       2   3.0
148       1   2.0
47        4   4.0
301      16   8.0
507       1   3.0
Root Mean Squared Error: 11.340674682559195
R^2 Score(coefficient of determination): 0.16924356555823605


## Random Forest with regression using PCA

In [61]:
# Build random forest using RandomForestRegressor
random_forest_model = RandomForestRegressor(n_estimators = 500, random_state = 1).fit(X_train,y_train)

# Perdict for test cases
random_forest_model_predictions = random_forest_model.predict(X_test)

# Create data frame for actual and predicted values
absenteeism_random_forest = pd.DataFrame({'actual': y_test, 'pred': random_forest_model_predictions})
print(absenteeism_random_forest.head())

# Calculate RMSE and R-squared value
print("Root Mean Squared Error: {}".format(
    str(calculate_root_mean_square_error(y_test, random_forest_model_predictions)))
)
print("R#^2 Score(coefficient of determination): {}".format(str(r2_score(y_test, random_forest_model_predictions))))

# print("Confusion matrix: {}".format(confusion_matrix(y_test, random_forest_model_predictions)))
# print("Accuracy score is: {}".format(accuracy_score(y_test, random_forest_model_predictions) * 100))

     actual       pred
328       2   3.125000
148       1   3.430967
47        4   2.846533
301      16  21.980000
507       1   5.848000
Root Mean Squared Error: 11.028831792183091
R#^2 Score(coefficient of determination): 0.2143032533699346
